<a href="https://colab.research.google.com/github/ttogle918/AI_practice/blob/main/ttogle918/search_bert/workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 준비

In [ ]:
!pip install kss

In [ ]:
!pip install pytorch-transformers transformers
!pip install sentence-transformers
!pip install -q konlpy tqdm

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import f1_score
from scipy import stats
import time
import matplotlib.pyplot as plt
from datetime import datetime
import math
import random
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.neighbors import NearestNeighbors
import requests
from bs4 import BeautifulSoup
import re

from sentence_transformers import SentenceTransformer, util

from transformers import AutoTokenizer, PreTrainedTokenizerFast, PreTrainedTokenizer
from transformers import pipeline

In [ ]:
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

## 데이터셋 Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try :
  with open('/content/drive/MyDrive/비상바로가기/최지현/폴더 업로드 방침 등등.txt', 'r') as f :
    print(f.readlines()[0])
    print('ok')
  data_path = f'/content/drive/MyDrive/비상바로가기/최지현'
except FileNotFoundError as e :
  print(e)
  neo_path = "/content/drive/MyDrive/== 비상교육 ==/최지현"
  data_path = f'{neo_path}'



ok


In [ ]:
import pandas as pd
df = pd.read_csv(f'{data_path}/<온리원>/생성된데이터/onlyone_question_temp.csv')
print(df.columns)
df.head()

In [ ]:
df_question = pd.read_csv(f'{data_path}/<온리원>/생성된데이터/only_questions.csv')
print(df_question.columns)
df_question.head(3)

In [ ]:
df_question = df_question.drop(columns=['답변키', '답변자키', '답변자명', '답변제목', '답변내용', '답변등록일'])
df_question.head(2)

# FilterClass

In [ ]:
class FilterClass() :
  def __init__(self, sentiment_model_path, eth_model_path, fword_path) :
    self.pipe_sentiment = pipeline(model=sentiment_model_path, tokenizer=AutoTokenizer.from_pretrained(sentiment_model_path, truncation_side='right')) # 감성 분석    # 
    self.pipe_eth = pipeline(model=eth_model_path, tokenizer=AutoTokenizer.from_pretrained(eth_model_path, truncation_side='right')) # 윤리적 문장인지 분류

    # 욕설 단어를 포함한 게시글인지 확인하는 조건 load
    self.fword = []
    with open(fword_path, 'r') as f :
      for line in f.readlines() :
        self.fword.append(line.replace('\n', '').strip())

  def run(self, df) :
    # 욕설 문장 제거
    df, df_fword = self.remove_fword(df)
    print(f'욕설이 포함된 질문 list. 총 {len(df_fword)}개\n')
    for q in df_fword.질문 :
      print(q)
    
    # 부정 문장 제거
    df, df_negative = self.remove_negative(df)
    print(f'부정적인 질문 list. 총 {len(df_negative)}개\n')
    for q in df_negative.질문 :
      print(q)

    # 비윤리적 문장 제거
    df, df_unethical = self.remove_uneth(df)
    print(f'비윤리적 문장인 질문 list. 총 {len(df_unethical)}개\n')
    for q in df_unethical.질문 :
      print(q)
    return df, df_fword, df_negative, df_unethical

  def remove_fword(self, df) :
    df = df.get()   # 질문
    df['have_fword'] = df['질문'].apply(lambda q : sum([q.count(fw) for fw in self.fword]) )
    df_fword = df[df['have_fword'] > 0].copy()
    df = df.drop(df[df['have_fword'] > 0].index)
    df = df.drop(columns=['have_fword'])
    return df, df_fword

  def remove_uneth(self, df) :

    # tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512,'return_tensors':'pt'}
    result = self.pipe_eth(df['질문'].tolist(), batch_size=64, truncation=True)
    

    df[['unethical', 'score']]= pd.DataFrame.from_records(result)
    df_unethical = df[df['unethical'] == 1].copy()
    df = df.drop(df[df['unethical'] == 1].index)#.reset_index(drop=True)
    df = df.drop(columns=['unethical'] )#.reset_index(drop=True)
    return df, df_unethical

  def remove_negative(self, df) :

    result = self.pipe_sentiment(df['질문'].tolist(), batch_size=64, truncation=True)
    df[['sentiment', 'score']]= pd.DataFrame.from_records(result)
    df_negative = df[df['sentiment'] == 1].copy()
    df = df.drop(df[df['sentiment'] == 1].index)#.reset_index(drop=True)
    df = df.drop(columns=['sentiment'] )#.reset_index(drop=True)
    return df, df_negative
  # 다른 모델 사용 시
  # def sentiment_test(self, model, test_loader, device):
  #   model.to(device)
  #   model.eval()
  #   preds = []

  #   with torch.no_grad():
  #       for sentence in tqdm(iter(test_loader)):
  #           sentence = sentence.to(device)
  #           logit = model(sentence)
  #           preds += logit.argmax(1).detach().cpu().numpy().tolist()
  #   return preds

# CustomDataset : dataset

In [ ]:
type(df[['질문']]), type(df['질문'])

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [ ]:
class CustomDataset():
    def __init__(self, df):
      # case 1. 질문, 답변 모두 들어오는 경우 ==> 기존 질문
      # case 2. 질문만 입력되는 경우 ==> 새 질문(추론)
      self.sentence = self.preprocessing(df)

      if '답변내용' in df.columns :
        self.answer = df['답변내용']
        self.name = df['질문자명']
    
    def get(self) :
      return self.sentence

    def __getitem__(self, index):
      if index == '질문' :
          return self.sentence
      elif index == '답변내용' :
          return self.answer
      elif index == '질문자명' :
          return self.name
      
      if self.answer is not None:
          return self.sentence.iloc[index], self.answer.iloc[index], self.name.iloc[index]
      else:
          return self.sentence.iloc[index]

    def __len__(self):
      return len(self.sentence)

    def preprocessing(self, df_temp) :
      df_temp['질문'] = df_temp['질문'].apply(lambda x : self.replace_continue_string(x))    # 반복 문자 없애기
      df_temp['질문'] = df_temp['질문'].apply(lambda x : BeautifulSoup(x,"html5lib").get_text())    # html tag 없애기
      df_temp['질문'] = df_temp['질문'].apply(self.remove_emoji)   # 이모지 없애기
      df_temp['질문'] = df_temp['질문'].apply(lambda x : x.replace('\xa0', ''))
      return df_temp[['질문']]

    def replace_continue_string(self, s) :
      dic = self.continue_string(s)
      for key in dic :
        s = s.replace(key*dic[key], key*2)
      return s
      
    def remove_emoji(self, text) :
      only_BMP_pattern = re.compile("["
              u"\U00010000-\U0010FFFF"  #BMP characters 이외
                                "]+", flags=re.UNICODE)
      return only_BMP_pattern.sub(r'', text) # no emoji

    def continue_string(self, s) :
      bef_string = ''
      cnt, i = 1, 0
      dic = {}
      while i < len(s) :
        if bef_string == s[i] :
          cnt += 1
        else :
          bef_string = s[i]
          cnt = 1
        if cnt > 2 :
          dic[bef_string] = cnt
        i += 1
      return dic

# CustomClass : 유사한 문장 얻기

In [ ]:
# pandas의 dataframe, df['질문']이 input으로 들어감 ==> apply를 쓰려고 pandas 형태로 넣음
# base_question에 필요한 것
class CustomClass() :
  def __init__(self, similarity_model_path, base_questions) :
    # 유사도 측정한 모델
    self.embedder = SentenceTransformer(similarity_model_path)    
    # 기존 문장 전처리
    # 기존 질문에 대해 filtering 수행 ==> 밖에서 수행해야함!!
    # 기존 문장 임베딩
    self.corpus_embeddings = self.embedder.encode(base_questions)

  # 새 질문 필터링 후, 유사도 찾기
  def run(self, df, threshold=-1) :
    # 유사한 질문 문장 index 얻기
    print('similarity')
    if threshold >= 1 :   # 1이상이면 k개 선택하라는 의미
      top_k = min(threshold, len(self.corpus_embeddings))
      top_results = self.get_similar_sentences(df, top_k)    # score, idx
    else :    # 1 미만이라면 ( 원래는 -1 ~ 1 까지의 값 가능 ) 유사도가 임계값 이상인 데이터만 사용하라는 의미
      top_results = self.get_similar_sentences_by_threshold(df, threshold)    # score, idx

    # 유사한 질문 문장과 그 답변 print
    return top_results

  # 임계값으로 구분 
  def get_similar_sentences_by_threshold(self, df, threshold) :
    ret_dict = {}
    queries = df['질문'].tolist()
    query_embeddings = self.embedder.encode(queries)

    for i, query in enumerate(queries):
      cos_scores = util.cos_sim(query_embeddings[i], self.corpus_embeddings)[0]
      check = ( cos_scores>=threshold ).nonzero(as_tuple=True)[0]
      ret_dict[query] = {'score' : cos_scores[check], 'idx' : check }
    return ret_dict

  def get_similar_sentences(self, df, top_k) :
    ret_dict = {}
    queries = df['질문'].tolist()
    query_embeddings = self.embedder.encode(queries)

    for i, query in enumerate(queries):
      cos_scores = util.cos_sim(query_embeddings[i], self.corpus_embeddings)[0]
      top_results = torch.topk(cos_scores, k=top_k)
      ret_dict[query] = {'score' : top_results[0], 'idx' : top_results[1] }
    return ret_dict

  def make_dataloader(self, dataset, tok_model, batch_size, s='test') :
    global tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tok_model)
    dataloader = DataLoader(
        dataset,
        batch_size =batch_size,
        sampler = SequentialSampler(dataset),
        collate_fn = self.custom_collate_fn
        # num_workers = 1
    )
    print(f'batch_size : {batch_size}')
    return dataloader
  
  def custom_collate_fn(self, batch):
    input1_list = []
    for _input1 in batch:
        input1_list.append(_input1)
    
    tensorized_input = tokenizer(
        input1_list,
        add_special_tokens=True,
        padding="longest",
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    return tensorized_input

In [ ]:
def print_similar(df, top_results) :
  for query in df['질문'].tolist() :
    print(f'\n질문 : {query}')

    # for score, idx in zip(top_results[query]['dist'], top_results[query]['idx']):
    if 'answer_score' in top_results[query] :
      
      score, answer_score, idx = top_results[query]['score'], top_results[query]['answer_score'], top_results[query]['idx']
      for i in range(len(score)) :
        print(f"{base_dataset[idx[i]][0]}", "(Score: {:.4f})".format(score[i]))
        print(f"{base_dataset[idx[i]][1]}", "(Score: {:.4f})".format(answer_score[i]))
        print('\n')
  
def preprocessing(df) :
  df['질문'] = df['질문'].apply(lambda x : BeautifulSoup(x,"html5lib").get_text())
  # df['질문'] = df['질문'].apply(lambda x : ''.join(dict.fromkeys(x) ) )
  return df

# de_identifyClass

In [ ]:
import kss

class de_identifyClass() :
  # df column에 이름 내역 포함되어있어야 함
  def __init__(self) :
    reg_ptrn = "(\d{6}[ ,-]-?[1-4]\d{6})|(\d{6}[ ,-]?[1-4])"
    addr_ptrn = "([가-힣]+[도시구군]\s)+(\(?[가-힣]+\d?[읍로길동가](\d[가-힣]\s?)?\)?\s?([0-9]+)?)+-?[0-9]+?\s?([가-힣]+\s{0,2}아파트)?([가-힣]{2,10}빌)?\s?(([0-9]{1,4}[동])?\s?([0-9]{1,4}[호])?)?"
    # addr_ptrn = "([가-힣]+[도시구군]\s)+(\(?[가-힣]+\d?[읍로길동가](\d[가-힣]\s?)?\)?\s?([0-9]+)?)+-?[0-9]+([가-힣]아파트)?([가-힣]빌)?\s?([0-9]+[동호]\s?)?"
    mail_ptrn = "^[\d][A-Za-z0-9!-_\.]+@[A-Za-z]+\.[A-Za-z]+"
    account_ptrn = "^(\d{1,})(-(\d{1,})){1,}"
    phone_ptrn = "\d{2,3}[ ,-]?\d{2,4}[ ,-]?\d{4}"
    self.embedding_dim = 512   # bert-base 임베딩 벡터 크기
    
    self.reg_prog = re.compile(reg_ptrn)
    self.addr_prog = re.compile(addr_ptrn)
    self.mail_prog = re.compile(mail_ptrn)
    self.account_prog = re.compile(account_ptrn)
    self.phone_prog = re.compile(phone_ptrn)

  def de_identify_private_info(self, content) :
    content = self.phone_prog.sub('(휴대폰번호)', content)
    content = self.reg_prog.sub('(주민번호)', content)
    content = self.mail_prog.sub('(이메일)', content)
    content = self.account_prog.sub('(계좌번호)', content)
    return content

  def de_identify_address(self, sentence) :
    sentence = self.addr_prog.sub('(주소)', sentence)

    return sentence

  def de_identify_name(self, sentence, name) :
    # 이름 익명화
    # 질문 = 질문.apply(작성자명없애기.)
    sentence = sentence.replace(name[1:], '온리원')
    sentence = sentence.replace(name, '온리원')
    return sentence

  def run(self, sentence, name) :
    # 주소 : NER 추가 혹은 정규표현식 주석 제거 ==> 두가지 모두 사용, 합집합(더 넓은 범위) 적용
    
    # 정규표현식으로 패턴 매칭
    sentence = self.get_private_info(sentence)
    # 이름 수정 ==> "온리원"이~
    sentence = self.de_identify_name(sentence, name)
    return sentence

  # NER 주소 
  def get_entity_per_word(self, str_dic_list) :
    print("get_entity_per_word")
    result_li = []
    for dic_list in str_dic_list :
      result_li.append(self.get_location_index(dic_list))
    return result_li

  def get_location_index(self, dic_list) :
    if len(dic_list) == 0 :
      return []
    start_idx, end_idx = -3, -3
    isLoc = False
    ret_list = []   # stack ==> .pop()
    word = ''
    for dic in dic_list :
      # 이어질 때
      if end_idx + 2  >= dic['start'] :    # 띄어쓰기, 마침표 때문에 +2까지 허용
        end_idx = dic['end']    # end index로 문장 길이 파악
        word += dic['word']
        if dic['entity'] == 'LOC-B' :
          isLoc = True

      else :    # 이어지지 않을 때
        # 이전 개체가 loc이라면 저장
        if isLoc == True :
          # 주소인 location일 때는 7글자가 넘을 것이다. ==> (서울시) 중구 태평로1가 31, 중구 세종대로 (110)
          if end_idx - start_idx > 7 :
            ret_list.append([start_idx, end_idx, word])
          isLoc = False

        start_idx = dic['start']
        end_idx = dic['end']
        word = dic['word']
        
        if dic['entity'] == 'LOC-B' :
          isLoc = True

    if isLoc == True :
      if end_idx - start_idx > 7 :
        ret_list.append([start_idx, end_idx, word])
    return ret_list

  # token의 수가 512를 넘을 때 런타임 에러가 남. 
  # 개인정보 삭제가 목표이기 때문에 모든 문장을 확인할 필요가 있음.
  # 문장 분리한 뒤 주소 확인, 다시 합침
  def find_location(self, sentence) :
    loc_li = []
    try :
      # loc_li : 찾아낸 주소를 [start_idx, end_idx, 주소명]으로 담고있는 리스트
      loc_li = self.get_location_index(self.ner(sentence))
    except RuntimeError :   # token의 길이가 임베딩 최대 길이보다 클 때 나타나는 오류
      try :
        # 1. 문장 split
        splited_sent = kss.split_sentences(sentence)
        # 최대한 많은 문장을 담아야 주소를 잘 인식함 ==> embedding 차원의 수를 넘지 않는 최대 길이의 문장 얻기
        start_idx, sent_length, temp_sent = 0, 0, []
        for i, sent in enumerate(splited_sent) :
          sent_length += len(sent)
          if sent_length > self.embedding_dim :
            temp_sent.append(''.join(splited_sent[start_idx:i]))   # 이번 sent는 포함하지 않음
            sent_length, start_idx = len(sent), i
            # 한 문장이 임베딩 차원보다 클 때(한 문장 : 문장 분리기로 나눈 결과)
            if sent_length > self.embedding_dim :
              temp_sent.append(''.join(splited_sent[start_idx:i][:self.embedding_dim]))   # 이번 sent는 포함하지 않음
              sent_length, start_idx = len(sent)-self.embedding_dim, i
              
        temp_sent.append(''.join(splited_sent[start_idx:]))

        result = []
        for i, li in enumerate(self.ner(temp_sent)) :
          loc_li = self.get_location_index(li)
          
          while len(loc_li) > 0 :
            start_idx, end_idx, word = loc_li.pop()
            temp_sent[i] = temp_sent[i].replace(temp_sent[i][start_idx:end_idx], '(주소)')
        return ''.join(temp_sent)
      except :
        print(sentence)
        print(splited_sent)
        print(sent_length)
    try :
      while len(loc_li) > 0 :
        start_idx, end_idx, word = loc_li.pop()
        sentence = sentence.replace(sentence[start_idx:end_idx], '(주소)') 
    except :
      print(sentence)
    return sentence


# test : 질문 = 강의명 + 질문제목 + 질문 내용

In [ ]:
df.columns

In [ ]:
df = df.fillna('')
df_question = df_question.fillna('')
df.isna().sum()

In [ ]:
df['질문'] = df['강의명'] + df['질문제목'] + df['질문내용']
df_question['질문'] = df_question['강의명'] + df_question['질문제목'] + df_question['질문내용']

base_dataset = CustomDataset(df[['질문', '답변내용', '질문자명']])

In [ ]:
sentiment_model_path = 'baikal-nlp/dbert-sentiment'
eth_model_path = 'baikal-nlp/dbert-eth2'
fword_path =f'{data_path}/<필터링단어>/fword_list.txt'

filter = FilterClass(sentiment_model_path, eth_model_path, fword_path)

In [ ]:
df, df_fword, df_negative, df_unethical = filter.run(base_dataset)

In [ ]:
df

In [ ]:
# df.to_csv(f'{data_path}/요약:질문 검색/after_filtering.csv', index=False)

In [ ]:
similarity_embedder = f'{data_path}/요약:질문 검색/튜닝된모델'

start_time = time.time()
model = CustomClass(similarity_embedder, df['질문'].tolist())
end_time = time.time()
print('초기화에 걸린 시간 : ', end_time - start_time)

In [ ]:
df_question['질문'] = df_question['질문'].astype(str)
df_question['질문여부'] = df_question['질문'].apply(lambda x : x.find('질문'))
df_question = df_question[df_question['질문여부'] > -1]

In [ ]:
len(df_question)

## 코사인유사도 값이 0.7(임계값) 이상일 때

In [ ]:
start_time = time.time()
# top_results = model.run(df_question.iloc[:100].질문.to_frame(), 2)    # 가장 비슷한 2개의 질문 출력 시 사용
top_results = model.run(df_question.iloc[:100].질문.to_frame(), 0.7)
end_time = time.time()

print('추론에 걸린 시간 : ', end_time - start_time)

In [ ]:
def print_similar(df, top_results) :
  for query in df['질문'].tolist() :
    print(f'\n질문 : {query}')

    for score, idx in zip(top_results[query]['score'], top_results[query]['idx']):
      print(f"{de_identification.run(df[idx]['질문'], df[idx]['이름'])}")
      print(f"(Score: )".format(score))
      print(f"{df[idx][1]}", "(Score: {:.4f})".format(score))
      print(f"(Score: )".format(score))
      print('\n')
  return top_results

In [ ]:
start_time = time.time()
de_identification = de_identifyClass(df, top_results)
# idx, top_results, 
print_similar(df, top_results)
end_time = time.time()
print('익명화+결과도출에 걸린 시간 : ', end_time - start_time)